In [98]:
from ipycanvas import Canvas
from ipywidgets import Output
import numpy as np
from ipywidgets import interact
import cv2 as cv
import va

In [108]:
w,h,bs = 562,562,12
mtx = np.zeros((w,h),dtype=int)

@interact(brush_size=(2,32),width=(100,1024),height=(100,1024),RunNeutalNet=False)
def set_param(brush_size,width,height,RunNeutalNet):
    global w,h,bs
    bs = brush_size
    w = width
    h = height
    if RunNeutalNet :        
        net = cv.dnn.readNet('minst/model.onnx')
        net.setInput(cv.dnn.blobFromImage(mtx)) # image.depth() != blob_.depth() DOMANDA
        out = net.forward()
        classe = np.argmax(out)
        va.show(mtx)
        print(f"classe = {classe}")

interactive(children=(IntSlider(value=17, description='brush_size', max=32, min=2), IntSlider(value=562, descr…

In [109]:
canvas = Canvas(width=w, height=h)
canvas.put_image_data(mtx, 0, 0)
mtx = np.zeros((w,h),dtype=np.uint16)
out = Output()
reg = False

@out.capture()
def handle_mouse_move(x, y):
    global reg
    if reg :
        y, x = int(y), int(x)
        bs2 = bs//2
        mtx[y-bs2:y+bs2+1,x-bs2:x+bs2+1] = 255
        canvas.put_image_data(mtx, 0, 0)

@out.capture()
def handle_mouse_down(x, y):
    global reg
    reg = True

@out.capture()
def handle_mouse_up(x, y):
    global reg
    reg = False
    
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_up(handle_mouse_up)

display(out)
canvas

Output()

Canvas(height=562, width=562)